# TensorFlow Convolutional Neural Network for Image Classification

In [10]:
from __future__ import print_function
import numpy as np
import time
import math
import random
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

## Imports
Here, we import keras, the model we are going to use (a sequential model), three different types of layers namely convolution layer, max-pooling layer and dense layers. We will also import activation functions and flattening operation from the layers module.

In [11]:
import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten, Dropout
from keras.optimizers import RMSprop

## Load Data
There is a lot to building a succesful model. Loading datasets, formatting them, presenting them to the model in batches are all part of a successul machine learning algorithm. Here we show how the specific data we are using is loaded, and formatted in the Keras library.

In [12]:
#from keras.datasets.cifar import load_batch #cifar doesn't exist in Keras Python 3
from keras.datasets.cifar10 import load_data as load_data_cifar10
from keras import backend as K

# Additional helper function cobbled together from scouring the internet
import pickle
def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = pickle.load(f, encoding='bytes')
    d_decoded = {}
    for k, v in datadict.items():
        d_decoded[k.decode('utf8')] = v
    datadict = d_decoded
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0, 1, 2, 3).astype('float')
    Y = np.array(Y)
    return X, Y

def load_data():
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    path = './cifar-10-batches-py'

    num_train_samples = 50000

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        (x_train[(i - 1) * 10000: i * 10000, :, :, :],
         y_train[(i - 1) * 10000: i * 10000]) = load_CIFAR_batch(fpath) # load_batch(fpath) load_batch doesn't exist in keras.dataset.cifar10

    fpath = os.path.join(path, 'test_batch')
    x_test, y_test = load_CIFAR_batch(fpath) # load_Batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if K.image_data_format() == 'channels_last':
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)

    return (x_train, y_train), (x_test, y_test)

## Exploring the dataset
First we need to figure out our image size and number of categories(or classes) we are trying model. Fill in the following parameters.

In [13]:
#  The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
def params():
    num_classes = 10
    in_shape = (32,32,3)
    batch_size = 32
    epochs = 10
    return [num_classes, in_shape, batch_size, epochs]

In [14]:
[num_classes, in_shape, batch_size, epochs] = params()

In [15]:
(x_train, y_train), (x_test, y_test) = load_data()
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Building the Model
Here you are going to build the CNN model. Provide the implementation for ConvConvMaxpoolDropout followed by ConvConvMaxpoolDropout and a classification module of DenseDropDense layers

In [16]:
#Three different types of layers namely convolution layer (Conv2D), max-pooling layer, and dense layers. We also use the flattening, activation and dropout
# I see now the ask above is saying to do a Conv2d - Conv2d - MaxPool - Dropout set layers + regularization, and then another set for the second one, then a third one with flatten, dense, dropout dense

def build_model(in_shape):
    model = Sequential()

    # ConvConvMaxpoolDropout
    model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2)) # Drop out 20% of the nodes

    # ConvConvMaxpoolDropout with more filters (I just doubled the input)
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3)) # Drop out 30% of the nodes

    # DenseDropDense
    model.add(Flatten())
    model.add(Dense(units=128, activation='relu')) # units equal to previous layer (64+64)
    model.add(Dropout(0.5)) # Dropout 50% of the nodes
    model.add(Dense(units=10, activation='softmax')) # 10 output classes, use softmax function for activation
    return model

In [17]:
# Putting everything together. Build your model, initiate RMSprop optimizer, and compile it with categorical_crossentropy
def train_cifar():
    model = build_model(in_shape)
    opt = RMSprop(learning_rate=0.0001, rho=0.9, momentum=0.0) #weight_decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_test, y_test), shuffle=True)
    return model

model = train_cifar()

Epoch 1/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 282s 180ms/step - accuracy: 0.2248 - loss: 2.0914 - val_accuracy: 0.4311 - val_loss: 1.6005
Epoch 2/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 326s 182ms/step - accuracy: 0.4051 - loss: 1.6437 - val_accuracy: 0.4957 - val_loss: 1.4004
Epoch 3/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 327s 185ms/step - accuracy: 0.4657 - loss: 1.4812 - val_accuracy: 0.5352 - val_loss: 1.2914
Epoch 4/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 318s 182ms/step - accuracy: 0.5091 - loss: 1.3724 - val_accuracy: 0.5769 - val_loss: 1.2072
Epoch 5/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 325s 184ms/step - accuracy: 0.5384 - loss: 1.2959 - val_accuracy: 0.6009 - val_loss: 1.1476
Epoch 6/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 322s 184ms/step - accuracy: 0.5680 - loss: 1.2267 - val_accuracy: 0.6021 - val_loss: 1.1118
Epoch 7/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 319s 182ms/step - accuracy: 0.5910 - loss: 1.1611 - val_accuracy: 0.6428 - val_loss: 1.0192
Epoch 8/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 319s 180ms/step - ac

In [18]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.7437 - loss: 0.7548
Test loss: 0.759059727191925
Test accuracy: 0.7426000237464905


# Analysis

1. The model is structured above with multiple Convolutional (Conv2D) layers of filters, two at each "section", with ReLU activations with dropout, followed by flattening, adding a dense layer, and dropout. There's a total of 4 convolutional layers with max pooling done between every set of 2, and then a dense layer at the end with 128 units (I initially thought I was connecting to the 64+64 of the previous layer). Same padding keeps the same shape between Convolutional layers, but with that I'm not sure how projecting the 3x3 into the greater filter space of the next layer, then replacing every 2x2 area with it's max value really works out to a good number of filters in the next layer. I just took a naive approach to the calculations and decided to double the values between the "sections" of the question.
2. I tried several different values for our main hyperparameters such as learning_rate, weight_decay, etc. and found that 0.0001 gave significantly better results than 0.01 and 0.00001 over 10 epochs. I extrapolated this to 25 epochs (which took quite awhile to run) and saw increasing accuracy. As accuracy began to increase, the error term started flattening and I gained less accuracy over time as expected. A 72.72% accuracy isn't terrible though for this sort of model, although probably not perfect.

In [ ]:
%%shell

jupyter nbconvert --to html /content/Lab04.ipynb